## 231205.ipynb 작업 내용 요약
- HN__4_8__19__selected.csv: 새로운 raw 데이터 셋 생성함.  
- 231204_v4 내용 반영
    1) dtype = object 제외 (해당사항 없음.)  
    2) 9, 99, 999 인 녀석들(모름/무응답) null 처리  
    3)  mh_PHQ_S 는 null값인데, 각 문항 응답인 BP_PHQ_1 ~ 9의 값이 있는 경우, 더해서 mh_PHQ_S에 넣어주기 -> 유효한 mh_PHQ_S를 최대한 늘리기 위한 작업  
    4) 결측값 처리 (1) - row 기준 결측 비율이 30% 넘는 행 제외   
    5) `depressed` 변수 생성  
	    1) 기준 (1) - `mh_PHQ_S >= 10`  
	    2) 기준 (2) - `BP_PHQ_9`에 1,2,3점으로 대답한 경우 (자살 시도 또는 생각)    
    6) 여기까지 `depressed` 비율 확인  
	    1) `depressed == 0`: 80,909  
	    2) `depressed == 1`: 1,399  
    7) 잠재적으로 우울한 사람 drop  
	    1) `DF2_pr == null & mh_PHQ_S == null`  
	    2) `DF2_pr != 8 or mh_PHQ != NaN`  
    8) 결측값 처리 (2) - column 기준 결측 비율이 10% 넘는 열 제외  
	    1) shape: (79743, 94)  
    9) 결측값 처리 (3) - fillna  
	    1) num_features 결측값 처리 - KNN  
	    2) onehot_features, label_features 결측값 처리 - 최빈값(most frequent values)  
	    3) `BP6_10 == NaN or BP6_31 == NaN` 이면 해당 행 drop  
		    1) 두 변수 모두 1년간 자살 생각 및 자살 시도 여부를 예 아니오로 응답한 질문,  null 값 채우기에는 위험하기 때문에 일단 null rows drop, ###TODO 현재는 정확률을 일부로 높이기 위해 해당 변수를 X 피쳐에 포함 시킬 것. 추후에는 해당 변수를 아예 제거해야 정확한 실험 가능합니다.   
    10) 피쳐 그룹별 인코딩 작업  
	    1) numeric -> 인코딩 없이 사용, 단 scaling 필요함 (일단 MinMaxScaler 사용)  
	    2) onehot -> onehot 인코딩으로 진행 (categorical)  
	    3) label -> label 인코딩으로 진행 (ordinal)  
		    1. 비해당 값은 수동으로 적절한 라벨로 변경  
    11) Sampling  
	    1) Under sampling - random  
	    2) Over sampling - SMOTE  
    12) Output to csv  

In [8]:
import pandas as pd
import numpy as np

df_origin = pd.read_csv('../data/HN__4_8__19__selected.csv') #임시 -csv직접 올려서 사용함

### 데이터셋 설명
HN__4_8__19__selected.csv
- num_features = 연속형 숫자형 변수들
- onehot_features = 예/아니오 거나 응답별로 서로 순서/척도 관계 없이 정말 범주로만 나와있는 변수들 
- label_features = 적음/보통/많음 과 같이 척도 관계가 있는 변수들
- y_related = ["BP_PHQ_1","BP_PHQ_2","BP_PHQ_3","BP_PHQ_4","BP_PHQ_5","BP_PHQ_6","BP_PHQ_7","BP_PHQ_8","BP_PHQ_9","mh_PHQ_S","BP6_10","BP6_31","DF2_pr","DF2_pt"]

- 총 125개의 columns
- 4기 ~ 8기
- 만 19세 이상만 까지 처리해서 저장한 파일: HN__4_8__19__selected.csv

In [9]:
num_features = ["age",
"ainc",
"HE_PLS_15",
"HE_PLS_30",
"HE_mPLS",
"HE_sbp",
"HE_dbp",
"HE_ht",
"HE_wt",
"HE_wc",
"HE_glu",
"HE_HbA1c",
"HE_insulin",
"HE_chol",
"HE_HDL_st2",
"HE_TG",
"HE_LDL_drct",
"HE_ast",
"HE_alt",
"HE_HB",
"HE_HCT",
"HE_BUN",
"HE_crea",
"HE_WBC",
"HE_RBC",
"HE_Bplt",
"HE_Uacid",
"HE_Uph",
"HE_Usg",
"HE_Ucrea"]

In [10]:
onehot_features = ["region",
"sex",
"occp",
"marri_1",
"tins",
"npins",
"D_2_1",
"DI1_pr",
"DI1_pt",
"DI2_pr",
"DI2_pt",
"DI3_pr",
"DI3_pt",
"DJ4_pr",
"DJ4_pt",
"DE1_pr",
"DE1_pt",
"BH9_11",
"BH1",
"BH2_61",
"LQ4_00",
"LQ1_sb",
"LQ2_ab",
"AC1_yr",
"MH1_yr",
"EC_occp",
"EC_stt_1",
"EC_stt_2",
"EC_wht_0",
"BO1_1",
"BO2_1",
"BD1",
"BS8_2",
"BS9_2",
"LW_ms",
"LW_pr",
"LW_oc",
"HE_rPLS",
"HE_Unitr",
"HE_Upro",
"HE_Uglu",
"HE_Uket",
"HE_Ubil",
"HE_Ubld",
"HE_Uro",
"BM1_0",
"BM1_1",
"BM1_2",
"BM1_3",
"BM1_4",
"BM1_5",
"BM1_6",
"BM1_7",
"BM1_8",
"L_BR_TO",
"L_LN_TO",
"L_DN_TO",
"LK_LB_US",
"LK_LB_EF",
"live_t",
"educ",
"BO1",
"HE_obe",]

In [11]:
label_features = ["incm",
"ho_incm",
"incm5",
"ho_incm5",
"edu",
"cfam",
"house",
"D_1_1",
"DI3_2",
"BD1_11",
"BD2_1",
"BA2_12",
"BA2_13",
"BP1",
"BS3_1",
"BE3_31",
"BE5_1",
"OR1",
]

In [12]:
y_related =["BP_PHQ_1",
"BP_PHQ_2",
"BP_PHQ_3",
"BP_PHQ_4",
"BP_PHQ_5",
"BP_PHQ_6",
"BP_PHQ_7",
"BP_PHQ_8",
"BP_PHQ_9",
"mh_PHQ_S",
"BP6_10",
"BP6_31",
"DF2_pr",
"DF2_pt"]

In [13]:
df = df_origin[num_features+onehot_features+label_features+y_related]
df

,age,ainc,HE_PLS_15,HE_PLS_30,HE_mPLS,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,...,BP_PHQ_5,BP_PHQ_6,BP_PHQ_7,BP_PHQ_8,BP_PHQ_9,mh_PHQ_S,BP6_10,BP6_31,DF2_pr,DF2_pt
0,61.0,460.833333,NaN,37.0,NaN,137.5,82.5,179.9,86.7,103.7,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,0.0,0.0
1,57.0,460.833333,NaN,36.0,NaN,158.0,102.0,179.3,82.0,92.6,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,8.0,8.0
2,39.0,257.500000,NaN,35.0,NaN,104.5,67.5,159.9,59.0,79.3,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,8.0,8.0
3,19.0,257.500000,NaN,31.0,NaN,98.0,61.0,169.7,63.1,74.3,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,8.0,8.0
4,70.0,133.750000,NaN,NaN,59.0,105.5,69.0,171.4,55.8,72.7,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94114,74.0,58.333333,NaN,NaN,NaN,132.0,73.0,151.6,69.1,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,8.0,8.0,8.0
94115,25.0,58.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94116,36.0,200.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94117,34.0,200.000000,NaN,NaN,NaN,95.0,59.0,151.4,57.9,77.5,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,8.0,8.0


In [14]:
## 20231204_v4 반영
## ['LQ4_24', 'N_DT_DS', 'N_DUSUAL', 'N_DAY'] object 타입이라 뺌
## [9, 99, 999, 9999] 무응답 널처리

####### 얘네 빼도 되는지 확인 - object 타입인 녀석들 ####
# df = df.drop(columns=['LQ4_24', 'N_DT_DS', 'N_DUSUAL', 'N_DAY'], inplace=False)

####### 9, 99, 999 인 녀석들(모름/무응답) null 처리
df = df.replace([9, 99, 999, 9999], np.nan, inplace=False)

In [15]:
cond = df["BP_PHQ_1"].notna() & \
        df["BP_PHQ_2"].notna() & \
        df["BP_PHQ_3"].notna() & \
        df["BP_PHQ_4"].notna() & \
        df["BP_PHQ_5"].notna() & \
        df["BP_PHQ_6"].notna() & \
        df["BP_PHQ_7"].notna() & \
        df["BP_PHQ_8"].notna() & \
        df["BP_PHQ_9"].notna()

df[df["mh_PHQ_S"].isnull() & cond][["BP_PHQ_1", "BP_PHQ_2", "BP_PHQ_3", "BP_PHQ_4", "BP_PHQ_5", "BP_PHQ_6", "BP_PHQ_7", "BP_PHQ_8", "BP_PHQ_9"]].value_counts()

BP_PHQ_1  BP_PHQ_2  BP_PHQ_3  BP_PHQ_4  BP_PHQ_5  BP_PHQ_6  BP_PHQ_7  BP_PHQ_8  BP_PHQ_9
0.0       0.0       3.0       3.0       3.0       0.0       0.0       0.0       0.0         26
3.0       0.0       3.0       3.0       0.0       0.0       0.0       0.0       0.0         11
1.0       1.0       1.0       1.0       1.0       1.0       1.0       1.0       1.0          7
3.0       3.0       0.0       3.0       0.0       0.0       0.0       0.0       0.0          5
1.0       1.0       3.0       2.0       2.0       0.0       0.0       0.0       0.0          4
                                                                                            ..
          0.0       3.0       2.0       3.0       0.0       0.0       0.0       0.0          1
                              3.0       0.0       0.0       0.0       2.0       0.0          1
                                                  1.0       1.0       0.0       0.0          1
          1.0       0.0       2.0       2.0       1.0   

In [16]:
# mh_PHQ_S 는 null값인데, 각 문항 응답인 BP_PHQ_1 ~ 9의 값이 있는 경우, 더해서 mh_PHQ_S에 넣어주기
a = df[df["mh_PHQ_S"].isnull() & cond][["BP_PHQ_1", "BP_PHQ_2", "BP_PHQ_3", "BP_PHQ_4", "BP_PHQ_5", "BP_PHQ_6", "BP_PHQ_7", "BP_PHQ_8", "BP_PHQ_9"]]

b = a[a <= 3].sum(axis=1)
b[b >= 1] #125

df['mh_PHQ_S'] = b[b >=1]

In [17]:
### 결측값 처리 1
# row 기준 
threshold = 0.3 # threshold for null proportion
null_percent = df.isnull().mean(axis = 1) # 각 행에 대한 결측값 비율 계산
df = df[null_percent < threshold]
df

,age,ainc,HE_PLS_15,HE_PLS_30,HE_mPLS,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,...,BP_PHQ_5,BP_PHQ_6,BP_PHQ_7,BP_PHQ_8,BP_PHQ_9,mh_PHQ_S,BP6_10,BP6_31,DF2_pr,DF2_pt
0,61.0,460.833333,NaN,37.0,NaN,137.5,82.5,179.9,86.7,103.7,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,0.0,0.0
1,57.0,460.833333,NaN,36.0,NaN,158.0,102.0,179.3,82.0,92.6,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,8.0,8.0
2,39.0,257.500000,NaN,35.0,NaN,104.5,67.5,159.9,59.0,79.3,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,8.0,8.0
3,19.0,257.500000,NaN,31.0,NaN,98.0,61.0,169.7,63.1,74.3,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,8.0,8.0
4,70.0,133.750000,NaN,NaN,59.0,105.5,69.0,171.4,55.8,72.7,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94110,74.0,NaN,NaN,NaN,NaN,109.0,63.0,143.5,54.1,102.8,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,8.0,8.0
94113,75.0,58.333333,NaN,NaN,NaN,165.0,71.0,174.3,80.2,97.3,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,8.0,8.0,8.0
94114,74.0,58.333333,NaN,NaN,NaN,132.0,73.0,151.6,69.1,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,8.0,8.0,8.0
94117,34.0,200.000000,NaN,NaN,NaN,95.0,59.0,151.4,57.9,77.5,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,8.0,8.0


In [18]:
# 생성변수: `depressed` Define depressed (dependent variable)
cond1 = df['mh_PHQ_S'] >= 10 # mh_PHQ_S - total score 
cond2 = df['BP_PHQ_9'].isin([1, 2, 3]) # person who chose 1, 2, 3 in 9th question 

df[cond1 | cond2].drop_duplicates() # person with cond1 or cond2

,age,ainc,HE_PLS_15,HE_PLS_30,HE_mPLS,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,...,BP_PHQ_5,BP_PHQ_6,BP_PHQ_7,BP_PHQ_8,BP_PHQ_9,mh_PHQ_S,BP6_10,BP6_31,DF2_pr,DF2_pt
6015,52.0,820.000000,NaN,NaN,49.0,133.0,92.0,159.7,62.4,86.4,...,3.0,3.0,3.0,3.0,3.0,NaN,8.0,2.0,8.0,8.0
6042,53.0,698.333333,NaN,30.0,NaN,93.0,63.0,150.9,47.9,70.7,...,1.0,1.0,1.0,0.0,1.0,NaN,8.0,2.0,1.0,0.0
6055,27.0,250.000000,NaN,33.0,NaN,123.0,82.0,167.2,51.3,70.6,...,0.0,2.0,0.0,1.0,1.0,NaN,8.0,2.0,8.0,8.0
6082,43.0,170.000000,NaN,37.0,NaN,122.0,97.0,173.2,98.9,110.9,...,1.0,2.0,0.0,0.0,1.0,NaN,8.0,2.0,8.0,8.0
6144,27.0,530.000000,NaN,46.0,NaN,114.0,83.0,175.2,108.9,117.2,...,3.0,3.0,0.0,1.0,1.0,NaN,8.0,2.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49897,37.0,225.000000,NaN,NaN,NaN,118.0,75.0,170.1,68.0,78.0,...,2.0,2.0,1.0,0.0,1.0,NaN,8.0,8.0,1.0,0.0
49924,80.0,46.666667,NaN,NaN,NaN,149.0,59.0,148.5,46.4,69.1,...,0.0,0.0,0.0,0.0,2.0,NaN,8.0,8.0,8.0,8.0
49948,46.0,380.000000,NaN,NaN,NaN,110.0,69.0,174.4,71.8,74.8,...,3.0,1.0,1.0,0.0,1.0,NaN,8.0,8.0,8.0,8.0
49958,55.0,583.333333,NaN,NaN,NaN,120.0,82.0,171.3,61.5,79.9,...,0.0,2.0,0.0,0.0,1.0,NaN,8.0,8.0,8.0,8.0


In [19]:
# Create a new variable 'depressed' 
df.loc[cond1 | cond2, 'depressed'] = 1 # 두 조건 중 하나 이상을 만족하는 행에 'depressed' 변수에 1 값 
df.loc[~(cond1 | cond2), 'depressed'] = 0 # 두 조건 중 하나라도 만족하지 않는 행에 0 값

In [20]:
# Check distribution of depressed to see if the data is imbalanced
df.groupby('depressed').count() # groupby를 이용해 타겟 y 라벨의 클래스 별 개수를 확인

,age,ainc,HE_PLS_15,HE_PLS_30,HE_mPLS,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,...,BP_PHQ_5,BP_PHQ_6,BP_PHQ_7,BP_PHQ_8,BP_PHQ_9,mh_PHQ_S,BP6_10,BP6_31,DF2_pr,DF2_pt
depressed,,,,,,,,,,,,,,,,,,,,,
0.0,80909,80209,5370,9308,5912,79573,80391,80626,80765,80567,...,20642,20632,20638,20645,20635,245,80871,80833,80905,80905
1.0,1399,1397,0,262,102,1372,1387,1385,1396,1393,...,1392,1387,1393,1394,1399,93,1399,1394,1399,1399


In [21]:
# Remove people who might be depressed
# 1) DF2_pr 도 null값이고, mh_PHQ_S 도 null값인 경우 이 사람에 대한 우울증 관련 정보가 없기 때문에 데이터에서 drop 
df.drop(df[df['DF2_pr'].isna() & df['mh_PHQ_S'].isna()].index, inplace=True)

In [22]:
df.shape # Originally from 82308 rows × 125 columns to 82304 rows × 126 columns

(82304, 126)

In [23]:
# 2) DF2_pr == 8 (never been diagnosed by doctor) & mh_PHQ_S == NaN) 인 경우를 제외하려면?
# DF2_pr != 8 or mh_PHQ != NaN (전부 다 ~ 취하기)
df = df[(df['DF2_pr'] != 8 |  ~df['mh_PHQ_S'].isna())]
df.shape

(79743, 126)

In [24]:
### 결측값 처리 2
# Remove columns if percentage of NaN exceeds 0.1
# threshold = 0.3 # 이거로 하나, 0.1로 하나 큰 차이 x
threshold = 0.1 

null_percent = df.isnull().mean() # 각 열에 대한 결측값 비율 계산
selected_cols = null_percent[null_percent < threshold].index
df = df[selected_cols]
df.shape # (20492, 314)

(79743, 94)

In [25]:
df.columns # 'DF2_pr', 'DF2_pt', 'depressed'는 살아 있어야 함 
#TODO
# DF2_pr 변수 및 DF2_pt 변수 고려해줄 건지... 현 상태에서는 일단 제외하고 모델 돌려보는 걸 목표로 한다. 

Index(['age', 'ainc', 'HE_sbp', 'HE_dbp', 'HE_ht', 'HE_wt', 'HE_wc', 'HE_glu',
       'HE_chol', 'HE_HDL_st2', 'HE_TG', 'HE_ast', 'HE_alt', 'HE_HB', 'HE_HCT',
       'HE_BUN', 'HE_crea', 'HE_WBC', 'HE_RBC', 'HE_Bplt', 'HE_Uph', 'HE_Usg',
       'sex', 'occp', 'marri_1', 'tins', 'npins', 'D_2_1', 'DI1_pr', 'DI1_pt',
       'DI2_pr', 'DI2_pt', 'DI3_pr', 'DI3_pt', 'DJ4_pr', 'DJ4_pt', 'DE1_pr',
       'DE1_pt', 'BH9_11', 'BH1', 'BH2_61', 'LQ4_00', 'LQ1_sb', 'LQ2_ab',
       'EC_occp', 'EC_stt_1', 'EC_stt_2', 'BO1_1', 'BO2_1', 'BD1', 'BS8_2',
       'BS9_2', 'HE_rPLS', 'HE_Unitr', 'HE_Upro', 'HE_Uglu', 'HE_Uket',
       'HE_Ubil', 'HE_Ubld', 'HE_Uro', 'BM1_1', 'BM1_2', 'BM1_3', 'BM1_4',
       'BM1_5', 'BM1_6', 'BM1_7', 'BM1_8', 'live_t', 'educ', 'BO1', 'HE_obe',
       'incm', 'ho_incm', 'incm5', 'ho_incm5', 'edu', 'cfam', 'house', 'D_1_1',
       'DI3_2', 'BD1_11', 'BD2_1', 'BA2_12', 'BA2_13', 'BP1', 'BS3_1',
       'BE3_31', 'BE5_1', 'BP6_10', 'BP6_31', 'DF2_pr', 'DF2_pt', 'depressed'],


In [26]:
# df[['DF2_pr', 'DF2_pt']].isnull().sum() # 0, 0
df[['BP6_10', 'BP6_31']].isnull().sum() # 35, 75

# 두 변수 모두 1년간 자살 생각 및 자살 시도 여부를 예 아니오로 응답한 질문 
# null 값 채우기에는 위험하기 때문에 일단 null rows drop 
###TODO 현재는 정확률을 일부로 높이기 위해 해당 변수를 X 피쳐에 포함 시킬 것.
"""추후에는 해당 변수를 아예 제거해야 정확한 실험 가능합니다. """
df.drop(df[df['BP6_10'].isna() | df['BP6_31'].isna()].index, inplace=True)
# df.shape

In [27]:
df.groupby('depressed').count() # groupby를 이용해 타겟 y 라벨의 클래스 별 개수를 확인

,age,ainc,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,HE_glu,HE_chol,HE_HDL_st2,...,BA2_12,BA2_13,BP1,BS3_1,BE3_31,BE5_1,BP6_10,BP6_31,DF2_pr,DF2_pt
depressed,,,,,,,,,,,,,,,,,,,,,
0.0,78502,77832,77201,78003,78229,78363,78173,74775,76924,76918,...,78468,78459,78444,78465,78347,78404,78502,78502,78502,78502
1.0,1151,1150,1129,1141,1138,1149,1147,1016,1062,1062,...,1144,1144,1143,1143,1142,1151,1151,1151,1151,1151


In [28]:
df.isnull().sum().sort_values(ascending = False) # 결측치 개수 확인

EC_occp      7175
HE_Bplt      5132
BH9_11       5016
HE_BUN       4948
HE_alt       4185
             ... 
DE1_pr          0
DJ4_pt          0
DJ4_pr          0
sex             0
depressed       0
Length: 94, dtype: int64

In [29]:
# Consider Removed columns to create grouped features again
removed = ['HE_PLS_15', 'HE_PLS_30', 'HE_mPLS', 'HE_HbA1c', 'HE_insulin', 'HE_LDL_drct', 'HE_Uacid', 'HE_Ucrea']
num_features = [item for item in num_features if item not in removed]

removed = ['region', 'AC1_yr', 'MH1_yr', 'EC_wht_0', 'LW_ms', 'LW_pr', 'LW_oc', 'BM1_0', 'L_BR_TO', 'L_LN_TO', 'L_DN_TO', 'LK_LB_US', 'LK_LB_EF']
onehot_features = [item for item in onehot_features if item not in removed]

removed = ['OR1']
label_features = [item for item in label_features if item not in removed]

# removed = ["BP_PHQ_1","BP_PHQ_2","BP_PHQ_3","BP_PHQ_4","BP_PHQ_5","BP_PHQ_6","BP_PHQ_7","BP_PHQ_8","BP_PHQ_9","mh_PHQ_S"]
# label_features = [item for item in y_related if item not in removed]

In [30]:
### 결측값 처리 3 - fillna: KNN
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

X = df.drop('depressed', axis = 1) # DF2_pr 들어가 있는 상태임
X.shape # (79743, 94) 

###TODO 피쳐 별로 결측값 채우는 방법을 달리 할 수 있음 - 추가 EDA 작업
"""
1) num_features: numerical이니 KNN 써도 됨
2) onehot_features, label_features: 보통 1.0 단위로 값이 커짐. --> KNN으로 결측값을 채우면 1.0 단위로 값이 나와야 하는데 1.2 이렇게 채워짐...
-> 다른 방법으로 결측값 채워야 함 -> 최빈값?
"""

# 1) num_features 결측값 처리 - KNN
knn_imp = KNNImputer(n_neighbors=5)
X[num_features] = knn_imp.fit_transform(X[num_features])

# 2) onehot_features, label_features 결측값 처리 - 최빈값(most frequent values)
freq_imp = SimpleImputer(strategy = "most_frequent")
X[onehot_features] = freq_imp.fit_transform(X[onehot_features])
X[label_features] = freq_imp.fit_transform(X[label_features])

/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:767: Fu

In [31]:
X['BA2_12'].value_counts() # 소숫점 단위로 들어간 게 혹시 있는지 확인 -> 없음 good

BA2_12
5.0    34977
8.0    33847
4.0     6737
3.0     2764
2.0      997
1.0      331
Name: count, dtype: int64

In [32]:
# transform array(X) to dataframe
df_new = pd.DataFrame(X, columns=df.drop('depressed', axis = 1).columns) 
df_new.isna().sum().sort_values(ascending = False)

age       0
HE_Uro    0
live_t    0
BM1_8     0
BM1_7     0
         ..
DI1_pt    0
DI1_pr    0
D_2_1     0
npins     0
DF2_pt    0
Length: 93, dtype: int64

### 피쳐 그룹별 인코딩 작업
1. numeric -> 인코딩 없이 사용, 단 scaling 필요함 (일단 MinMaxScaler 사용)
2. onehot -> onehot 인코딩으로 진행 (categorical)
3. label -> label 인코딩으로 진행 (ordinal)

In [33]:
# 1. numeric # 1. numeric - scaling 
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler(feature_range=(0, 1))
df_new[num_features] = scaler.fit_transform(df_new[num_features])

/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:767: Fu

In [34]:
# 2. onehot -> onehot 인코딩으로 진행
df_new = pd.get_dummies(df_new, columns=onehot_features) # sklearn OneHotEncoder 사용도 가능함

In [35]:
# 3. label -> label 인코딩으로 진행 (ordinal)
# label features 중에서 비해당 처리
label_ans_8 = ["DI3_2", "BD1_11", "BD2_11", "BA2_12", "BA2_13", "BP1", "BS3_1","BE3_31", "BE5_1"] # BD2_11 - removed


# 'DI3_2' 변수의 값이 8인 경우, 해당 값을 0으로 바꾸기
df_new['DI3_2'] = df_new['DI3_2'].apply(lambda x: 3 if x == 8 else x)
df_new['BD1_11'] = df_new['BD1_11'].apply(lambda x: 1 if x == 8 else x)
# df_new['BD2_11'] = df_new['BD2_11'].apply(lambda x: 0 if x == 8 else x)
df_new['BA2_12'] = df_new['BA2_12'].apply(lambda x: 5 if x == 8 else x)
df_new['BA2_13'] = df_new['BA2_13'].apply(lambda x: 5 if x == 8 else x)
df_new['BA2_13'] = df_new['BA2_13'].apply(lambda x: 5 if x == 8 else x)
# df_new['BP1'] = df_new['BP1'].apply(lambda x: 5 if x == 8 else x)
df_new['BS3_1'] = df_new['BS3_1'].apply(lambda x: 0 if x == 8 else x)
# df_new['BE3_31'] = df_new['BE3_31'].apply(lambda x: 5 if x == 88 else x)
# df_new['BE5_1'] = df_new['BE5_1'].apply(lambda x: 5 if x == 88 else x)

In [36]:
df_new['DI3_2'].value_counts() # 라벨 인코딩 잘 되었는지 체크

DI3_2
3.0    78692
1.0      656
2.0      305
Name: count, dtype: int64

In [37]:
df_new.isna().any().any()

False

### Sampling
- Under: random
- Over: SMOTE

In [38]:
# under sampling.  OVER 만 하기엔 너무 많이 생성해야해서 under 선수행
from imblearn.under_sampling import RandomUnderSampler

X = df_new
y = df['depressed']

rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_under, y_under = rus.fit_resample(X, y)
X_under.shape, y_under.shape

/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be 

((12661, 224), (12661,))

In [39]:
# over sampling
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_over, y_over = smote.fit_resample(X_under, y_under)

X_over.shape, y_over.shape

/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be 

((23020, 224), (23020,))

In [40]:
y_over.value_counts()

depressed
0.0    11510
1.0    11510
Name: count, dtype: int64

In [41]:
X_over.to_csv('../data/HN_X_231205.csv', index=False)
y_over.to_csv('../data/HN_y_231205.csv', index=False)